In [67]:
# Add a log normal distribution for pertubing the hash needed
import numpy as np
from bisect import bisect

def create_block_hashes(state, params):

    # Get the baseline block difficulties without randomness
    prime_block_hashes = [state["Global Difficulty"] * params["Block Difficulty Multiples"]["Prime"]]
    region_block_hashes = [state["Global Difficulty"] * params["Block Difficulty Multiples"]["Region"]] * state["Regions"]
    zone_block_hashes = [state["Global Difficulty"] * params["Block Difficulty Multiples"]["Zone"]]* state["Regions"] * state["Zones per Region"]

    # Merge and order
    block_hashes = []
    while len(region_block_hashes) > 0:
        for _ in range(state["Zones per Region"]):
            block_hashes.append(zone_block_hashes.pop())
        block_hashes.append(region_block_hashes.pop())
    block_hashes.append(prime_block_hashes.pop())

    # Add in randomness
    block_hashes = np.array(block_hashes)
    mutlipliers = np.random.lognormal(0, .05, len(block_hashes))
    block_hashes = block_hashes * mutlipliers
    block_hashes = block_hashes.round().astype(int)

    # Get cumulative sum for later computation of how far the aggregate hash rate gets us
    block_hashes_cs = np.cumsum(block_hashes)

    return block_hashes, block_hashes_cs

def compute_progress(state, params, block_hashes, block_hashes_cs):
    print("-"*20+"Block Mining"+"-"*20)
    total_hashpower = state["Aggregate Hashpower"] * state["Target Time"]
    if  total_hashpower >= block_hashes_cs[-1]:
        print("Prime block is mined!")
        time_to_mine = block_hashes_cs[-1] / total_hashpower* state["Target Time"]
        
        mined_blocks = block_hashes
        block_hashes, block_hashes_cs = [], []
    else:
        print("Prime block not mined!")
        time_to_mine = state["Target Time"]
        i = bisect(block_hashes_cs, total_hashpower)
        mined_blocks = block_hashes[:i]
        if i > 0:
            block_hashes_cs[i:] -= block_hashes_cs[i-1]
            block_hashes, block_hashes_cs = block_hashes[i:], block_hashes_cs[i:]


    print("Time to mine = {}".format(time_to_mine))
    print("The following difficulties are emitted from mined blocks: {}".format(mined_blocks))
    print()
    return block_hashes, block_hashes_cs, time_to_mine

In [72]:
state = {"Global Difficulty": 1000,
         "Aggregate Hashpower": 500, # Per Second
         "Regions": 3,
         "Zones per Region": 3,
         "Target Time": 2}

params = {"Block Difficulty Multiples": {"Prime": 1,
                                         "Region": .5,
                                         "Zone": .25},
        "Block Hash Multiplier Lognormal Distribution": {"mu": 0, "std": .05}
        }

block_hashes = []
for _ in range(100):
    # If no left over blocks then start the next bunch of blocks
    if len(block_hashes) == 0:
        block_hashes, block_hashes_cs = create_block_hashes(state, params)

    # Compute the progress made
    block_hashes, block_hashes_cs, time_to_mine = compute_progress(state, params, block_hashes, block_hashes_cs)

    # Simple assumption, aggregate hash rate grows by 50 each epoch
    print("-"*20+"Aggregate Hash Change"+"-"*20)
    print("Dummy assumption, hash += 50")
    state["Aggregate Hashpower"] += 50
    print("New aggregate hash: {}".format(state["Aggregate Hashpower"]))
    print()

--------------------Block Mining--------------------
Prime block not mined!
Time to mine = 2
The following difficulties are emitted from mined blocks: [240 242 246]

--------------------Aggregate Hash Change--------------------
Dummy assumption, hash += 50
New aggregate hash: 550

--------------------Block Mining--------------------
Prime block not mined!
Time to mine = 2
The following difficulties are emitted from mined blocks: [497 255 245]

--------------------Aggregate Hash Change--------------------
Dummy assumption, hash += 50
New aggregate hash: 600

--------------------Block Mining--------------------
Prime block not mined!
Time to mine = 2
The following difficulties are emitted from mined blocks: [279 509 251]

--------------------Aggregate Hash Change--------------------
Dummy assumption, hash += 50
New aggregate hash: 650

--------------------Block Mining--------------------
Prime block not mined!
Time to mine = 2
The following difficulties are emitted from mined blocks: [26

In [4]:
# Bisection to find the blocks processed

In [5]:
# If leftover blocks, the values get scaled by the change in difficulty